In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from my_functions import *
from three_filter import three_filter_method

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_100deg_uniform.npy')
pm_data_noerr = np.load('npy/pm_LAE_100deg_uniform.npy')
f_name = 'Source_cats/Source_cat_LAE_100deg_uniform.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
N_nb = 5
ew0min = 10

z_idx = np.zeros(mock['redshift_Lya_Arr'].shape).astype(bool)

for nb_c in range(6, 20):
    z_nb = w_central[nb_c] / 1215.67 - 1

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
        )

    diff_err = (errors[nb_c]**2 + cont_err**2)**0.5
    fwhm = nb_fwhm(tcurves, nb_c, True)

    line = (
    (pm_data[nb_c] - cont_est > 3*diff_err)
    & (pm_data[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )

    z_idx[line] = True

fig, ax = plt.subplots(figsize=(12, 9))

hist_total, bins = np.histogram(mock['redshift_Lya_Arr'], bins=np.linspace(2.5, 3.2, 40))

hist, bins = np.histogram(
    mock['redshift_Lya_Arr'][z_idx], bins=np.linspace(2.5, 3.2, 40)
    )
ax.bar(
    [(bins[i] + bins[i + 1])/2 for i in range(len(bins) - 1)], hist/hist_total,
    width=bins[1]-bins[0]
)

for nb_c in range(6, 15):
    ax.axvline(x=w_central[nb_c] / 1215.67 - 1, color='red')
ax.set_xlabel('z', fontsize=15)
ax.set_ylim((0, 1))
plt.show()

print('TOTAL SELECTED: {0}/{1} ({2:0.1f}%)'.format(
    len(np.where(z_idx)[0]), len(z_idx), len(np.where(z_idx)[0])/len(z_idx) * 100
    ))

In [ ]:
def conf_matrix(line_Arr, z_Arr, nb_c):
    w_in = list(nb_fwhm(tcurves, nb_c))
    w_in.sort()
    w_in += np.array([-5, 5])
    z_in = np.array([w / 1215.67 - 1 for w in w_in])
    
    TP = len(np.where( line_Arr &  ((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])
    FP = len(np.where( line_Arr & ~((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])
    TN = len(np.where(~line_Arr & ~((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])
    FN = len(np.where(~line_Arr &  ((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])

    return np.array([[TP, FP], [FN, TN]])

In [ ]:
cm = np.zeros((2, 2))
n = 0
N_nb = 5
for nb_c in range(6, 15):
    z_nb = w_central[nb_c] / 1215.67 - 1

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
        )

    diff_err = (errors[nb_c]**2 + cont_err**2)**0.5
    fwhm = nb_fwhm(tcurves, nb_c, True)

    line = (
    (pm_data[nb_c] - cont_est > 3*diff_err)
    & (pm_data[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )

    cm += conf_matrix(line, mock['redshift_Lya_Arr'], nb_c)

fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, ax=ax, cbar=False)
ax.set_xlabel('REAL', fontsize=15)
ax.set_ylabel('SELECTED', fontsize=15)
ax.set_title('Stack', fontsize=20)
ax.set_xticklabels(['True', 'False'])
ax.set_yticklabels(['True', 'False'])

plt.show()

print('True Positive Rate: {0:0.1f}%'.format(100 * cm[0,0] / (cm[0,0] + cm[1,0])))
print('True Negative Rate: {0:0.1f}%'.format(100 * cm[1,1] / (cm[1,1] + cm[0,1])))
print('Contamination: {0:0.1f}%'.format(100 * cm[0,1] / (cm[0,0] + cm[1,0])))
print('Accuracy: {0:0.1f}%'.format(100 * (cm[0,0] + cm[1,1]) / (np.sum(cm))))

In [ ]:
cm = np.zeros((2, 2))
n = 0
ew0min = 20
for nb_c in range(6,15):
    NB = pm_data[nb_c]
    BB_LC = pm_data[-3]
    BB_LU = pm_data[-2]
    NB_err = pm_err[nb_c]
    BB_LC_err = pm_err[-3]
    BB_LU_err = pm_err[-2]
    t_NB = np.array(tcurves['t'][nb_c])
    t_BB_LC = np.array(tcurves['t'][-3])
    t_BB_LU = np.array(tcurves['t'][-2])
    w_NB = np.array(tcurves['w'][nb_c])
    w_BB_LC = np.array(tcurves['w'][-3])
    w_BB_LU = np.array(tcurves['w'][-2])
    w_EL = w_central[nb_c]

    F_EL, A, B, A_err, B_err = three_filter_method(
        NB, BB_LC, BB_LU, NB_err, BB_LC_err, BB_LU_err,
        t_NB, w_NB, t_BB_LC, t_BB_LU, w_BB_LC, w_BB_LU, w_EL
    )

    f_cont = A * w_EL + B
    f_cont_var = w_EL**2 * A_err**2 + B_err**2
    EW0 = F_EL / f_cont * 1215.67/w_EL

    line = (
        (pm_data[nb_c] - f_cont > 3*(pm_err[nb_c]**2 + f_cont_var) ** 0.5)
        & (EW0 > 10)
    )

    cm += conf_matrix(line, mock['redshift_Lya_Arr'], nb_c)

fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, ax=ax, cbar=False)
ax.set_xlabel('REAL', fontsize=15)
ax.set_ylabel('SELECTED', fontsize=15)
ax.set_title('3-filter', fontsize=20)
ax.set_xticklabels(['True', 'False'])
ax.set_yticklabels(['True', 'False'])
plt.show()

print('True Positive Rate: {0:0.1f}%'.format(100 * cm[0,0] / (cm[0,0] + cm[1,0])))
print('True Negative Rate: {0:0.1f}%'.format(100 * cm[1,1] / (cm[1,1] + cm[0,1])))
print('Contamination: {0:0.1f}%'.format(100 * cm[0,1] / (cm[0,0] + cm[1,0])))
print('Accuracy: {0:0.1f}%'.format(100 * (cm[0,0] + cm[1,1]) / (np.sum(cm))))

In [ ]:
N_nb = 5

diff_median_Arr = []
z_Arr = []
for nb_c in range(6, 15):
    cont_est, cont_err = stack_estimation(pm_data, pm_err, nb_c, N_nb, w_central)
    real_cont = NB_synthetic_photometry(mock['SEDs_no_line'], mock['w_Arr'],
        w_central[nb_c], nb_fwhm(tcurves, nb_c, True))

    w_in = list(nb_fwhm(tcurves, nb_c))
    w_in.sort()
    z_in = np.array([w / 1215.67 - 1 for w in w_in])
    line = (z_in[0] < mock['redshift_Lya_Arr']) & (mock['redshift_Lya_Arr'] < z_in[1])

    diff_median_Arr.append(np.median(cont_est[line] - real_cont[line]))
    z_Arr.append(w_central[nb_c] / 1215.67 - 1)

fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(z_Arr, diff_median_Arr, marker='.', linestyle='', markersize=15)
ax.set_xlabel('z', fontsize=15)
ax.set_ylabel('diff', fontsize=15)
plt.show()

In [ ]:
print(np.average(f_cont_var**0.5))
print(np.average(cont_err))